In [1]:
### only run once! sets up working directory to be /quality_issue_predictor
import os
os.chdir(os.getcwd()[:-34] + r'\quality_issue_predictor') 

In [2]:
import pandas as pd
import numpy as np
import os
import re
#import tensorflow as tf
import matplotlib.pyplot as plt

#import qa_productivity_tool.complaints as cmplnts
import qa_productivity_tool.nc_full as nc_full

Table of Contents <a name = "tableofcontents"></a>
1. [Data Cleaning/preparing](#datacleaning)
    1. [Archived Shop Order Cleanup](#socleanup)
    2. [Plant Star Cleanup](#plantstarcleanup)
    3. [Combine SO and Plantstar Date](#combinesoplantstar)
    4. [Quality Data injestion](#qualitydata)
2. [Models](#models)
    1. [K-means No NC data](#kmeansnonc)
    2. [K-means combinded with NC Data](#kmeanswithnc)
    3. [Decision Tree Models](#decisiontrees)
        1. [Adaboost](#adaboostresults)
        2. [Decision Tree Regressor](#dtrresults)
    4. [Dimensional Reduction Attempts](#dimensionalreduction)
        1. [PCA](#pca)

## Data Cleaning/Preparing

### Archived/Current Shop Order Clean up

In [4]:
df_arch = pd.read_excel(open('./raw_data/archived SO.xlsx','rb'))
df_curr = pd.read_csv(open('./raw_data/current_so.csv','rb'))

In [8]:
drop_cols = ['MQISS','Unnamed: 0', 'MOPNO'
df_curr.columns

Index(['Unnamed: 0', 'Shop Order', 'Component', 'MRdate', 'MQISS', 'Product',
       'BOM qty', 'BOM qty.1', 'MOPNO', 'Finished qty', 'MQREQ',
       'Requested qty', 'Lot Number', 'SOSTS', 'Date'],
      dtype='object')

In [5]:
cols = {'MORD':"Shop Order",'MPROD':"Component",'MAPRD':"Product",'MBOM':'BOM qty','SOLOT':'Lot Number','MRDTE':'MRdate',
       'SQFIN':'Finished qty'}
drop_cols = ['MQISS', 'MASTS','MOPNO']

In [6]:
df_arch = df_arch.rename(columns = cols)\
        .drop(columns = drop_cols)
df_arch.shape

(51587, 10)

In [7]:
df_arch.columns

Index(['Shop Order', 'Component', 'MRDTE', 'Product', 'BOM qty', 'SQFIN',
       'MQREQ', 'SQREQ', 'Lot Number', 'Date'],
      dtype='object')

In [ ]:
def lot_checker(lot_number):
    'checks if lot number matches YYMXXX63 format'
    if str(lot_number) == 'nan':
        return False
    return (re.compile('(:?[0-9]{2}[A-Ma-m]{1}[Oo0-9]{3}63[Xx]|[0-9]{2}[A-Ma-m]{1}[Oo0-9]{3}63)').match(lot_number) != None) *1
df['Lot Format Match'] = list(map(lot_checker,df['Lot Number']))
df = df[df['Lot Format Match'] == 1].drop(columns = 'Lot Format Match')

In [ ]:
def cleanup(frame):
    frame = frame.reset_index(drop=True)
    lot_number = frame.loc[0,'Lot Number']
    product = frame.loc[0,'Product']
    date = frame.loc[0,'Date']
    shop_order = str(frame.loc[0,'Shop Order'])
    df = pd.DataFrame()
    df.loc[lot_number,'Product'] = product
    df.loc[lot_number,"Date"] = date
    df.loc[lot_number,'Shop Order'] = shop_order
    for i,component in enumerate(frame.Component):
        df.loc[lot_number,component] = frame.loc[i,'BOM qty']
    return df.reset_index()\
            .rename(columns = {'index':'Lot Number'})


In [ ]:
list_of_frames = [cleanup(frame) for lot, frame in df.groupby(by = 'Lot Number')]
X = pd.concat(list_of_frames)\
            .fillna(0)\
            .reset_index(drop = True)

In [ ]:
X.shape

In [ ]:
X.head()

In [ ]:
print(min(X.Date).strftime("%Y-%b-%d"))
print(max(X.Date).strftime("%Y-%b-%d"))

Why are there lots through Dec 2021?

In [ ]:
X.head()

In [ ]:
df.head()

### Plantstar Clean-up <a name = "plantstarcleanup"></a>
[return](#tableofcontents)

In [ ]:
plant_df = pd.read_excel(open("./raw_data/Plantstar.xlsx",'rb'))

In [ ]:
plant_df.head()

In [ ]:
cols = {'user_text_4':'Shop Order'}
plant_df = plant_df.rename(columns = cols)
plant_df.head()

In [ ]:
def plantstar_extract(shop_order):
    tools = list(plant_df['tool'][plant_df['Shop Order'] == shop_order].unique())
    machines = list(plant_df['mach_name'][plant_df['Shop Order'] == shop_order].unique())
    num_entries = len(plant_df['start_time'][plant_df['Shop Order'] == shop_order]) #related to how long SO ran?
    return tools, machines, num_entries

### Combine SO data and Plantstar Data <a name = "combinesoplantstar"></a>
[return](#tableofcontents)

In [ ]:
for i, shop_order in enumerate(X['Shop Order']):
    tools,machines,num_entries = plantstar_extract(shop_order)
    if num_entries == 0:
        X = X.drop(i)
        continue
    for t in tools:
        X.loc[i,t] = 1
    for m in machines:
        X.loc[i,m] = 1
    X.loc[i,'num_plantstar_entries'] = num_entries
X = X.fillna(0)\
    .reset_index(drop= True)
X.head()

### Quality Records Data <a name = "qualitydata"></a>
[return](#tableofcontents)

In [ ]:

nc_data = nc_full.NC_Full()
nc_data.mostrecentreport(og_data_path = os.getcwd()+r'\raw_data')

In [ ]:
nc_data = nc_data.run_report()

In [ ]:
nc_data['Lot Number']

In [ ]:
for i, lot_number in enumerate(X['Lot Number']):
    for j, nc_lot_nums in enumerate(nc_data['Lot Number']):
        if lot_number in nc_lot_nums:
            X.loc[i,'NC'] = 1
            break


In [ ]:
X = X.fillna(0)

In [ ]:
sum(X.NC)

In [ ]:
X.head()

## Models <a name = 'models'></a>
[return](#tableofcontents)

### Kmeans clustering no NC data <a name = 'kmeansnonc'></a>
[return](#tableofcontents)

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
for col in X.columns[4:]:
    if str(col) == 'nan':
       X =  X.drop(columns =col)

In [ ]:
X_cluster = X[X.columns[4:]].to_numpy()

In [ ]:
kmeans = KMeans(n_clusters=8, random_state=0).fit(X_cluster)

In [ ]:
len(kmeans.labels_)

In [ ]:
len(X_cluster)

In [ ]:
X["labels"] = list(kmeans.labels_)

In [ ]:
X[X.labels == 7]

In [ ]:
for group, frame  in X.groupby(by = 'labels'):
    print(group, str(len(frame)))

### Combine k-Means results with NCs <a name = 'kmeanswithnc'></a>
[return](#tableofcontents)

In [ ]:
for group, frame  in X.groupby(by = 'labels'):
    print(group, sum(frame.NC)/len(frame)*100)


In [ ]:
X[X.labels == 7]

## Run K-Means with NC data

In [ ]:
X = X.drop(columns = ['labels'])
X_cluster = X[X.columns[4:]].to_numpy()

In [ ]:
kmeans = KMeans(n_clusters=9, random_state=0).fit(X_cluster)

In [ ]:
X["labels"] = list(kmeans.labels_)

In [ ]:
for group, frame  in X.groupby(by = 'labels'):
    
    print(group, len(frame), sum(frame.NC),round(sum(frame.NC)/len(frame)*100,2))

In [ ]:
X.replace(0.0,np.nan,inplace = True)

In [ ]:
X[X.labels == 7].count().sort_values(ascending=False)[6:16]

In [ ]:
X[X.labels == 3].count()\
                .sort_values(ascending = False)[6:16]\
                .reset_index()

In [ ]:
for product,frame in X[X.labels == 1].groupby(by='Product'):
    print(product, np.nansum(frame.NC)/len(frame))

In [ ]:
X[X.labels == 1].count().sort_values(ascending = False)[6:16]

In [ ]:
X = X.replace(0.0,np.nan,inplace = True)

In [ ]:
import seaborn as sns

## Decision Tree Models <a name = decisiontrees></a>
[return](#tableofcontents)

In [ ]:
X_cluster.shape

In [ ]:
X = X_cluster[:,:-1]
y = X_cluster[:,-1]

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
X, y, test_size=0.33, random_state=42)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor

In [ ]:
rng = np.random.RandomState(1)
regr_1 = DecisionTreeRegressor(max_depth=4)
regr_2 = AdaBoostRegressor(DecisionTreeRegressor(max_depth=10),
                          n_estimators=300, random_state=rng)

In [ ]:
import pickle

In [ ]:
filename = 'ada_boost_v1.sav'
ada_boost_v1 = pickle.dump(regr_2,open(filename,'wb'))

In [ ]:
from joblib import dump, load
dump(regr_2, 'ada_boost_v1.joblib') 

In [ ]:
regr_1.fit(X_train, y_train)
regr_2.fit(X_train,y_train)

In [ ]:
y_1 = regr_1.predict(X_test)
y_2 = regr_2.predict(X_test)

In [ ]:
os.getcwd()

In [ ]:
pd.DataFrame(X_train).to_csv('./training_data_v1.0/X_train.csv')

In [ ]:
pd.DataFrame(X_test).to_csv('./training_data_v1.0/X_test.csv')
pd.DataFrame(y_train).to_csv('./training_data_v1.0/y_train.csv')
pd.DataFrame(y_test).to_csv('./training_data_v1.0/y_test.csv')

### AdaBoost Results <a name = "adaboostresults"></a>
[return](#tableofcontents)

In [ ]:
from sklearn.metrics import precision_recall_curve


In [ ]:
precision, recall, thresholds = precision_recall_curve(y_test,y_2)

In [ ]:
plt.plot(thresholds,precision[:-1],label = 'Precision')
plt.plot(thresholds,recall[:-1],label = 'Recall')
plt.title('AdaBoost: Precision Recall Curve')
plt.xlabel('Threshold')
plt.legend()
plt.show()

In [ ]:
#thresholds
print(5*'-'+' Balanced Precision and Recall Model '+5*'-')
test_threshold = np.round(0.49714286,4)
print(f'percent of lots indicted = {round(100*len(y_2[y_2>=test_threshold])/len(y_2),2)}%')
print(f'recall (NC reduction) = {round(100*recall[np.where(np.round(thresholds,4) == test_threshold)][0],2)}%')
print(f'total number of predicted NCs = {len(y_2[y_2>=test_threshold])}')
print(f'True Positives = {sum(y_test[np.where(y_2>=test_threshold)])}')
print(f'precision = {round(100*precision[np.where(np.round(thresholds,4) == test_threshold)][0],2)}%')

In [ ]:
pd.DataFrame(thresholds).to_csv('./training_data_v1.0/example_thresholds.csv')

In [ ]:
pd.DataFrame(recall[1:]).to_csv('./training_data_v1.0/example_recall.csv')

In [ ]:
pd.DataFrame(precision[:-1]).to_csv('./training_data_v1.0/example_precision.csv')

In [ ]:
print(f'Number of SOs = {len(y_2)}')
print(f'Number of NCs = {sum(y_test)}')
print(f'Percent of SO with NCs in Test data = {round(100*sum(y_test)/len(y_2))}%')

### Decision Tree Regressor results <a name = dtrresults></a>
[return](#tableofcontents)

In [ ]:
precision, recall, thresholds = precision_recall_curve(y_test,y_1)

In [ ]:
plt.plot(thresholds,precision[:-1],label = 'Precision')
plt.plot(thresholds,recall[:-1],label = 'Recall')
plt.title('Precision Recall Curve')
plt.legend()
plt.show()

In [ ]:
thresholds

In [ ]:
precision[:-1]

In [ ]:
recall[:-1]

In [ ]:
y_1[y_1>=1]

### Nearest Neighbors <a name = nn></a>
[return](#tableofcontents)

In [ ]:
from sklearn import neighbors

In [ ]:
n_neighbors = 15
clf = neighbors.KNeighborsClassifier(n_neighbors, weights='distance')
clf.fit(X_train,y_train)

In [ ]:
nn_results = clf.predict(X_test)

In [ ]:
nn_precision, nn_recall, nn_thresholds = precision_recall_curve(y_test,nn_results)

In [ ]:
plt.plot(nn_thresholds,nn_precision[:-1],label = 'Precision')
plt.plot(nn_thresholds,nn_recall[:-1],label = 'Recall')
plt.title('NN: Precision Recall Curve')
plt.xlabel('Threshold')
plt.legend()
plt.show()

In [ ]:
nn_precision

## Dimensional Reduction <a name = "dimensionalreduction"></a>
[return](#tableofcontents)

### PCA <a name = pca></a>
[return](#tableofcontents)

In [ ]:
from sklearn import decomposition

In [ ]:
pca = decomposition.PCA(n_components=3)
pca.fit(X_test)

In [ ]:
X_pca = pca.transform(X_test)

In [ ]:
PCA_ADA_model = AdaBoostRegressor(DecisionTreeRegressor(max_depth=8),
                          n_estimators=300, random_state=rng)

In [ ]:
PCA_ADA_model.fit(X_train,y_train)

In [ ]:
y_pca_ada = PCA_ADA_model.predict(X_test)

In [ ]:
precision_pca, recall_pca, thresholds_pca = precision_recall_curve(y_test,y_pca_ada)

In [ ]:
plt.plot(thresholds_pca,precision_pca[:-1],label = 'Precision')
plt.plot(thresholds_pca,recall_pca[:-1],label = 'Recall')
plt.title('AdaBoost: Precision Recall Curve')
plt.xlabel('Threshold')
plt.legend()
plt.show()

In [ ]:
precision_pca[:-1]

In [ ]:
thresholds_pca

In [ ]:
test_threshold = np.round(0.47771836,4)
print(f'recall = {round(recall_pca[np.where(np.round(thresholds_pca,4) == test_threshold)][0],4)}')
print(f'precision = {round(precision_pca[np.where(np.round(thresholds_pca,4) == test_threshold)][0],4)}')
print(f'number of predicted NCs = {len(y_2[y_2>=test_threshold])}')
print(f'True Positives = {sum(y_test[np.where(y_2>=test_threshold)])}')